In [ ]:
#Correr en caso de no tener instalado
#!pip install emoji

#### Imports

In [1]:
import tweepy
import requests
import pandas as pd
import requests
import re
import emoji
from nltk import regexp_tokenize

## Conexión a la API de Twitter  via tweeppy y pedido de tuits

In [2]:

bearer_token= 'AAAAAAAAAAAAAAAAAAAAABO9iAEAAAAAC5GhM%2Fhjh64ff0YOMj4LaMk%2FtZo%3DzyVBVtgRRmbgK0HorXBDaUgHsLQcI2HLoTKME1shnD6fTWqyYH'
consumer_key = 'vmLmpiab4InZz4eHadryMwTRv'
consumer_secret =  '6J4aRsZQhf4ObZw3uboCMQyzzOmM3V7kkra4wXmOv7iKogpIse'
access_token = '359951893-XSPRBK4nVYT664gEQglScMBLcfFJLFSEjPlw0Sl2'
access_token_secret ='qRAblEDl2awmw0NdyGToQOG5pxwylS3dvpvaDNFt8mrnz'


        
client = tweepy.Client( bearer_token=bearer_token, 
                        consumer_key=consumer_key, 
                        consumer_secret=consumer_secret, 
                        access_token=access_token, 
                        access_token_secret=access_token_secret, 
                        return_type = requests.Response,
                        wait_on_rate_limit=True)

query = 'Milei -is:retweet lang:es '

# get max. 100 tweets
tweets = client.search_recent_tweets(query=query,max_results=10)



# Save data as dictionary
tweets_dict = tweets.json() 

# Extract "data" value from dictionary
tweets_data = tweets_dict['data'] 

# Transform to pandas Dataframe
df = pd.json_normalize(tweets_data)

#Link para armar una buena query
# https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/5-how-to-write-search-queries.md

In [3]:
df

,edit_history_tweet_ids,id,text
0,[1589339845771800577],1589339845771800577,@DamianFresolone Lo e escuchado hablar de cast...
1,[1589339717644218368],1589339717644218368,Encuentra las diferencias 😁\n\nJavier Milei #M...
2,[1589339444473401344],1589339444473401344,Que bárbaro lo que causo Milei en San Juan !\n...
3,[1589339415737794563],1589339415737794563,@LlanoMechi @JMilei Que manga de oportunistas ...
4,[1589339346897088512],1589339346897088512,Gracias a nueva tasa Milei le pagaremos más gu...
5,[1589339201832505344],1589339201832505344,Es mi Lamentable deber informar que Javier Mil...
6,[1589339145662795777],1589339145662795777,"El merluzo y su 68% de DESAPROBACIÓN, qué sent..."
7,[1589339139010281472],1589339139010281472,@shacco85 @lilianaf523 Venias bien y luego te ...
8,[1589339088074985473],1589339088074985473,@SergioChouza La esperanza es Milei. Cristina ...
9,[1589339020332789760],1589339020332789760,@mgrealpolitik @martintetaz Que es una boludez...


## Funciones de preprocesamiento antes de clasificar

In [4]:
 sgs = ["!","¡","#","$","%","&","(",")","*","+",",","-",".","/",":",";","<","=",">","?","@","[","]","^","_","`","{","|","}","~","]","´"]

def minusculizar(corpus = list()):
  """
  Func Minusculizar: Esta función se encarga de minusculizar cada oración de nuestro corpus.
  
  Params:
    - Corpus (list): Lista con todas las oraciones
  Return
    - Corpus (list): lista con todas las oraciones minusculizadas
  """

  corpus = [x.lower() for x in corpus]
  return corpus

def stopwords(corpus = list()):
  """
  Func Stopwords: Esta función se encarga de elimiar toda stopwords de cada oración en nuestro corpus.
  
  Params:
    - Corpus (list): Lista con todas las oraciones
  Return:
    - Corpus (list): lista con todas las oraciones libre de stopwords
  """

  data=[]
  for sentence in corpus:
    sentence = [x for x in sentence.split() if x not in sws]
    sentence= " ".join(sentence)
    data.append(sentence)

  return data


def convertir_emojis(corpus = list()):
  """
  Func Convertir_Emojis: Esta función se encarga de ubicar cada emoji de cada oración y reemplazarlo por el código que lo representa.

  Params:
    - Corpus (list): lista con todas las oraciones
  Return:
    - Corpus (list): lista con todas las oraciones donde sus emojis se ven reemplazados por sus codigos
  """    
  corpus = [emoji.demojize(sentence) for sentence in corpus]
  return corpus

def remover_regx(corpus = list()):
  """
  Func Remover_regx: Esta función se encarga de aplicar muchas expresiones regulares para limpiar las oraciones de aquellos signos o patrones
  no deseados. Entre estos están los @ con el nombre de usuario, las url del tipo https
  """
  data=[]
  for sentence in corpus:
    sentence = [re.sub("@\w+", "",x) for x in sentence.split()]
    sentence= " ".join(sentence)
    
    URL_REGEX = r"""((?:(?:https|ftp|http)?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|org|uk)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|uk|ac)\b/?(?!@)))"""
    sentence = re.sub(URL_REGEX,"",sentence)
    sentence = re.sub(r'[^\w\s]','', sentence)
    #sentence = [palabra for palabra in regexp_tokenize(sentence, pattern='\w+')]
    #sentence= " ".join(sentence)
    data.append(sentence)
  return data

def check_lenght(lista = list()):
  data=[]
  for tupla in lista:
    if len(tupla[0].split()) < 510:
      data.append(tupla)
  return data

def preprocesador(df):
  corpus=list(df.text)
  corpus = minusculizar(corpus)
  corpus = remover_regx(corpus)
  corpus = convertir_emojis(corpus)
  df = pd.DataFrame(corpus,columns =['sentence'])
 
  return df

### Llamado a la funcion

In [5]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
df_1=preprocesador(df)


In [6]:
df_1

,sentence
0,lo e escuchado hablar de casta política alguien le puede avisar al sr milei que el también forma parte que hace referencia de si mismo
1,encuentra las diferencias javier milei merluzoinepto alerta rechazoagenda2030 mario kreutzenberger mesacentral jadue cachureos pailita puerto montt andrés godoy teleton2022
2,que bárbaro lo que causo milei en san juan
3,que manga de oportunistas ahora se cuelgan de milei porque les conviene aunque en el fondo crean que es un loco
4,gracias a nueva tasa milei le pagaremos más guita a la policía aeroportuaria ojalá hagan algún curso de buenos modales ye idiomas son lamentables es un aeropuerto no un partido de futbol
5,es mi lamentable deber informar que javier milei ha fallecido qepd
6,el merluzo y su 68 de desaprobación qué sentirá realmente javier milei merluzoinepto alerta rechazoagenda2030 mario kreutzenberger mesacentral jadue cachureos pailita
7,venias bien y luego te fuiste al pasto creyendo que milei es la solución milei es funcional a los k
8,la esperanza es milei cristina si no pagará a sus militantes habría que ver con la crisis económica que hay y la pérdida de salario real quien quisiera ir a escucharla decir boludeces sin contar que siguen adoctrinando con la doctrina justicialista
9,que es una boludez dolarizar es con milei


## Carga de los tuits en la plataforma de clasificacion

In [7]:
url = 'https://rutbmzlxuu8vw3w-kgf771u4knq202wa.adb.sa-saopaulo-1.oraclecloudapps.com/ords/pln/tweets/insertar'

for tuit in df_1.sentence:
    tuit=tuit.encode('utf-8')
    x = requests.post(url, data =tuit)

    print(x.text)


200

200

200

200

200

200

200

200

200

200

